# UDACITY Designing Your First Workflow - Step Functions

## Step Functions & SageMaker

In the prior exercises, we've been working with many small services. This can be overwhelming for a data scientist that wants to establish a consistent methodology for handling data. Step Functions is an orchestration service that can allow us to utilize SageMaker in a methodical and consistent way. Step Functions also integrates with Lambda, which can allow us to potentially automate our entire machine learning pipeline end-to-end. Let's get a handle on what a 'step' in a step function looks like.

In this exercise, you will create a preprocessing step and a training step. Then you will create a step function to chain the two steps.

## Exercise: Grant Permissions and install packages.

Attach the IAMFullAccess and the StepFunctionsFullAccess polices to your SageMaker execution role.

In [3]:
%%bash
pip install "setuptools<66"
pip install stepfunctions

━━━━━━━━━━━ 1.2/1.2 MB 81.4 MB/s eta 0:00:00 ? eta -:--:--
?25hInstalling collected packages: setuptools
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.8.2
    Uninstalling setuptools-75.8.2:
      Successfully uninstalled setuptools-75.8.2
  Using cached stepfunctions-2.3.0.tar.gz (67 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
r/anaconda3/envs/python3/lib/python3.10/site-packages (from stepfunctions) (6.0.2)
s3transfer<0.12.0,>=0.11.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages (from boto3>=1.14.38->stepfunctions) (0.11.2)
cker in /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages (from sagemaker>=2.1.0->stepfunctions) (7.1.0)
3/lib/python3.10/site-packages (from sagemaker>=2.1.0->stepfunctions) (0.115.8)
b-metadata<7.0,>=1.4.0 in /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages (from sagemaker>=2.1.0->stepfunctions) (6.11.0)
/an

## Exercise: Fill out preprocessing step.

The 'step' interface is designed to be quite similar to the Preprocessing Job in lesson 2. The main difference between these is the ability of a 'step' to interface with other steps. Given the successful outcome of a single step, the next step specified in a workflow will automatically continue. In our case, a training step will launch given the successful outcome of a preprocessing step. The preprocessing step has been encoded for you. Upload the preprocessing code 'HelloBlazePreprocess.py' and the zipped dataset 'reviews_Musical_Instruments_5.json.zip' to s3, and fill out the constants in the code below. 

Code below is the preprocessing step. Fill in the constants in the code.

In [4]:
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from stepfunctions.steps.sagemaker import ProcessingStep
import sagemaker

role = get_execution_role()

PREPROCESSING_JOB_NAME = "test-job-preprocess"
input_data = 's3://stepfunctions16/stepfuncexcercise/Toys_and_Games_5.json.zip'
input_preprocessing_code = 's3://stepfunctions16/stepfuncexcercise/HelloBlazePreprocessLambda.py'
sess = sagemaker.Session()

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.large',
                                     instance_count=1)


processed_data_train = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_train_scikit')
processed_data_test = "{}{}/{}".format("s3://", sess.default_bucket(), 'hello_blaze_test_scikit')

inputs=[ProcessingInput(source=input_data, destination='/opt/ml/processing/input', input_name = 'input-1'),  ProcessingInput(source=input_preprocessing_code , destination='/opt/ml/processing/input/code', input_name = 'code')]


outputs=[ProcessingOutput(source='/opt/ml/processing/output/train', destination=processed_data_train, output_name = 'train_data'), ProcessingOutput(source='/opt/ml/processing/output/test', destination=processed_data_test, output_name = 'test_data')]


processing_step = ProcessingStep(
    "SageMaker pre-processing step 4",
    processor=sklearn_processor,
    job_name=PREPROCESSING_JOB_NAME,
    inputs=inputs,
    outputs=outputs,
    container_entrypoint=["python3", "/opt/ml/processing/input/code/HelloBlazePreprocess.py"],
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/03/25 15:37:03] INFO     Found credentials from IAM Role:                                   ]8;id=704459;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=501939;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
print(" ".join([processed_data_train, processed_data_test]))
print(inputs)

s3://sagemaker-us-east-1-620425758198/hello_blaze_train_scikit s3://sagemaker-us-east-1-620425758198/hello_blaze_test_scikit
[<sagemaker.processing.ProcessingInput object at 0x7f8a9ecc6680>, <sagemaker.processing.ProcessingInput object at 0x7f8aa15eb7c0>]


## Exercise: Fill out Training Step

Upon the success of the preprocessing step, we wish to execute a training step. A training step is defined below. Fill the constants in the code.

In [6]:
from stepfunctions.steps.sagemaker import TrainingStep
import boto3

WORKFLOW_OUTPUT = "s3://stepfunctions16/stepfuncexcercise/workflow_output"
TRAINING_JOB_NAME = "training-job-1"

region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    region=region_name, framework="blazingtext", version="latest"
)

helloBlazeEstimator = sagemaker.estimator.Estimator(
    container,
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size=30,
    max_run=360000,
    input_mode="File",
    output_path=WORKFLOW_OUTPUT,
    sagemaker_session=sess,
)

helloBlazeEstimator.set_hyperparameters(mode='supervised')

training_step = TrainingStep(
    "SageMaker Training Step",
    estimator=helloBlazeEstimator,
    data={"train": sagemaker.TrainingInput(processed_data_train, content_type="text/plain"), "validation": sagemaker.TrainingInput(processed_data_test, content_type="text/plain")},
    job_name=TRAINING_JOB_NAME,
    wait_for_completion=True,
)

## Exercise: Create Workflow & Execute It. 

To link the steps, you'll need to create a role that is capable of doing so. Go to IAM and create a Step Functions role, and attach the CloudWatchEventsFullAccess and SageMakerFullAccess policies. Once done, make use of the above steps to create a workflow. Quick debugging tip: jobs must have a unique name; you'll need to rename job names when debugging. Consider creating a method that will dynamically create unique job names! 

In [7]:
from stepfunctions.steps import Chain
from stepfunctions.workflow import Workflow

workflow_role = "arn:aws:iam::620425758198:role:"
arn:aws:iam::565094796913:role/step-function-execution-role

workflow_graph = Chain([processing_step, training_step])
workflow = Workflow(
    name="SageMakerProcessingWorkflow",
    definition=workflow_graph,
    role=workflow_role,
)

workflow.create()

execution = workflow.execute(
    inputs={
        "PreprocessingJobName": PREPROCESSING_JOB_NAME,  # Each pre processing job (SageMaker processing job) requires a unique name,
        "TrainingJobName": TRAINING_JOB_NAME  # Each Sagemaker Training job requires a unique name,       
    }
)

execution_output = execution.get_output(wait=True)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   role=workflow_role,                                                                     │
│   11 )                                                                                           │
│   12                                                                                             │
│ ❱ 13 workflow.create()                                                                           │
│   14                                                                                             │
│   15 execution = workflow.execute(                                                               │
│   16 │   inputs={                                                                                │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/stepfunctions/workflow/stepfu │
│ nctions.py:205 in create                                                                         │
│                                                                                                  │
│   202 │   │   │   return self.state_machine_arn                                                  │
│   203 │   │                                                                                      │
│   204 │   │   try:                                                                               │
│ ❱ 205 │   │   │   self.state_machine_arn = self._create()                                        │
│   206 │   │   except self.client.exceptions.StateMachineAlreadyExists as e:                      │
│   207 │   │   │   self.state_machine_arn = self._extract_state_machine_arn(e)                    │
│   208 │   │   │   logger.error("A workflow with the same name already exists on AWS Step Funct   │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/stepfunctions/workflow/stepfu │
│ nctions.py:213 in _create                                                                        │
│                                                                                                  │
│   210 │   │   return self.state_machine_arn                                                      │
│   211 │                                                                                          │
│   212 │   def _create(self):                                                                     │
│ ❱ 213 │   │   response = self.client.create_state_machine(                                       │
│   214 │   │   │   name=self.name,                                                                │
│   215 │   │   │   definition=self.definition.to_json(pretty=self.format_json),                   │
│   216 │   │   │   roleArn=self.role,                                                             │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py:569 in     │
│ _api_call                                                                                        │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_nam

You can track the outcome of this workflow through a custom UI that gets generated! Check it out!

In [ ]:
execution.render_progress()